In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np

In [8]:
dataset = pd.read_csv('D:\STUDIES\BTP\Data From Param\submit\graphsage\data.csv')
Data1Hr = dataset[dataset['GENERATION_TIME']<'2012.02.01 01:00']
data = Data1Hr[['LATITUDE','LONGITUDE', 'OZONE_PPB']]

In [9]:
normalizeData = (data - data.mean(axis = 0)) / data.std(axis = 0)
npData = normalizeData.to_numpy(dtype = float)
print(normalizeData)

     LATITUDE  LONGITUDE  OZONE_PPB
0    1.159329   0.217775  -2.267350
1    0.277968  -0.173864  -1.640807
2   -0.874582  -0.102657  -1.780039
3   -1.654248  -2.523696  -2.093310
4   -1.281364  -1.241969  -2.511005
..        ...        ...        ...
175  0.142374  -0.245071   1.317867
176  0.549156  -1.063952   0.830556
177  1.667807  -0.067053   1.178635
178  1.837300   0.680621   1.317867
179  1.261025  -0.316278   1.317867

[180 rows x 3 columns]


In [17]:
import random
trainRatio = 0.95
trainMask = random.sample(list(range(npData.shape[0])),int(npData.shape[0]*trainRatio))
trainMask.sort()
len(trainMask)

171

In [18]:
X = npData[:,:2]
Y = torch.tensor(npData[:,2], dtype = torch.float)
edgeList = []
Adj = np.zeros((X.shape[0],X.shape[0]))
dist = 0.015
size = X.shape[0]
for i in range(size):
    for j in range(size):
        if i!=j:
            d = (X[i][0]-X[j][0])**2 + (X[i][1] - X[j][1])**2
            if(d<dist):
                edgeList.append([i,j])
                Adj[i][j] = 1
    edgeList.append([i,i])
                
edges = np.array(edgeList)
edges = edges.T
edges

array([[  0,   1,   1,   2,   3,   4,   4,   5,   6,   6,   7,   7,   8,
          8,   9,   9,  10,  11,  11,  11,  11,  12,  12,  12,  13,  14,
         15,  16,  17,  17,  17,  17,  17,  18,  18,  19,  19,  20,  20,
         21,  21,  21,  21,  22,  23,  24,  24,  25,  25,  26,  26,  27,
         27,  27,  28,  28,  28,  28,  29,  30,  30,  31,  32,  33,  34,
         35,  36,  36,  36,  36,  37,  37,  37,  37,  37,  38,  39,  39,
         40,  40,  41,  41,  42,  42,  43,  43,  44,  44,  45,  45,  46,
         46,  47,  47,  48,  49,  50,  51,  52,  52,  53,  53,  54,  55,
         55,  56,  56,  57,  57,  57,  58,  59,  59,  60,  60,  61,  62,
         62,  63,  64,  64,  64,  65,  65,  66,  66,  67,  68,  69,  70,
         70,  70,  71,  71,  72,  72,  73,  73,  74,  74,  74,  74,  74,
         75,  75,  75,  75,  76,  76,  77,  77,  78,  78,  79,  80,  80,
         81,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  91,
         92,  92,  93,  94,  94,  95,  95,  96,  96

In [19]:
import torch
from torch_geometric.data import Data
x,y,e,trainMask = torch.tensor(X), torch.tensor(Y, dtype = torch.float), torch.tensor(edges, dtype = torch.long), torch.tensor(trainMask, dtype = torch.long)
graph = Data(x = x, edge_index = e, y = y, train_mask = trainMask)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
graph, graph.contains_self_loops(), graph.num_features

(Data(edge_index=[2, 320], train_mask=[171], x=[180, 2], y=[180]), True, 2)

In [21]:
#graph.x

In [22]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(2, 4)
        self.batch1 = nn.BatchNorm1d(4)        
        self.conv2 = GCNConv(4, 3)
        self.batch2 = nn.BatchNorm1d(3)
        self.conv3 = nn.Linear(3, 4)
        self.batch3 = nn.BatchNorm1d(4)
        self.conv4 = nn.Linear(4, 3)
        self.conv5 = nn.Linear(3, 1)

    def forward(self,data):
        x, edge_index = data.x.float(), data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.sigmoid(self.conv2(x, edge_index))
        self.saveLaplacianError(x,data)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.conv5(x)
        return torch.squeeze(x)
    
    def saveLaplacianError(self, x, data):
        A = torch_geometric.utils.get_laplacian(data.edge_index)
        Amat = torch.zeros(data.x.shape[0], data.x.shape[0])
        for i in range(len(A[1])):
            Amat[A[0][0][i].item(),A[0][1][i].item()] = A[1][i].item()
        Amat = Amat.float()
        P = torch.mm(torch.mm(torch.transpose(x.float(),0,1),Amat.float()),x.float())
        self.llLoss = torch.sum(torch.diag(P))

net = Net()
net = net.float()
net.train()

Net(
  (conv1): GCNConv(2, 4)
  (batch1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(4, 3)
  (batch2): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Linear(in_features=3, out_features=4, bias=True)
  (batch3): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Linear(in_features=4, out_features=3, bias=True)
  (conv5): Linear(in_features=3, out_features=1, bias=True)
)

In [23]:
import torch.optim as optim
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [27]:
def train():
    net.train()
    optimizer.zero_grad()
    output = net(graph)
    print(output.shape)
    loss = F.mse_loss(output[graph.train_mask],Y[graph.train_mask])
    print(type(loss))
    loss.backward()
    optimizer.step()
    return loss,output

In [28]:
out = torch.randn(5)
for epoch in range(1):
    loss, out = train()
    print(loss,i)
    i+=1

torch.Size([180])
<class 'torch.Tensor'>
tensor(0.8354, grad_fn=<MseLossBackward>) 755


In [433]:
net.eval()
net(graph),Y

(tensor([-2.1309e-01, -2.1309e-01, -5.5806e-02, -5.2054e-01, -2.1309e-01,
         -2.1309e-01, -6.2509e-01, -6.2509e-01, -6.2509e-01, -2.1309e-01,
          2.8082e-02, -2.1309e-01, -2.1309e-01, -2.1309e-01, -4.1778e-01,
         -6.2509e-01, -6.2509e-01, -2.1309e-01, -3.0953e-01, -2.4578e-01,
         -3.9029e-01, -5.6000e-01, -6.2509e-01, -6.2509e-01, -4.7669e-01,
          1.0085e-01, -2.1309e-01, -1.2410e-01, -5.6000e-01, -2.1309e-01,
         -5.5110e-01,  2.2002e-01, -2.1309e-01, -4.2201e-01, -5.2792e-01,
         -2.1309e-01, -5.6000e-01, -2.1309e-01, -5.8211e-01, -7.9064e-02,
          3.9919e-01, -2.6378e-03, -2.1309e-01, -2.6720e-01, -6.2509e-01,
         -2.1309e-01, -2.1309e-01, -5.7676e-01, -4.9495e-01, -6.2509e-01,
         -6.2509e-01, -2.1309e-01, -5.5110e-01, -5.7676e-01, -6.2509e-01,
         -6.2509e-01, -6.2509e-01, -6.2509e-01, -4.2092e-01, -3.2420e-01,
         -3.2420e-01,  2.9301e-01, -1.4416e-01, -2.1309e-01, -2.1309e-01,
         -2.1309e-01, -2.1309e-01,  1.

In [26]:
X.shape

(180, 2)

In [173]:
sum(abs(out-Y)>0.1)

tensor(78)

In [159]:
len(test)

171